# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS for 2 bedroom

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc
from AMI_BANDS_2021 import *

In [3]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',
             '0400118','0400119','0400120','0400121','0400122','0400123',
             '0400125','0400128','0400112','0400129']

#variables needed: household size NP, income INCP, & income adjustment factor
data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC'

In [4]:
#assuming July 2021 $1 for HHInc equivalent to estimate 2023 Inc equivalent
inflation_adjust = 1.11

In [5]:
#if there's time, it would be better to upload a dict of prices from MAG
#then generate the dictionary through calcs in python (vs. excel)
#flag for later

#Income to afford median household rent price + utilities by PUMA
#3 bedroom multifamily unit
inc_needed_mf = {'0400112':108100,'0400113':82800,'0400114':68770 ,'0400115':64285,
              '0400116':63250,'0400117':75900,'0400118':76475,'0400119':56925,
              '0400120':74750,'0400121':58650,'0400122':54970,'0400123':54970,
              '0400125':55154,'0400128':66700,'0400129':79350}

 
#Income to afford median household sales price by PUMA
#3 bedroom single-family unit
inc_needed_sf = {'0400112':110308,'0400113':83214,'0400114':67390,'0400115':63894,
              '0400116':56856,'0400117':65918,'0400118':70334,'0400119':50876,
              '0400120':79350,'0400121':62238,'0400122':52164,'0400123':68540,
              '0400125':55706,'0400128':72634,'0400129':80960}

In [6]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [7]:
df = get.get_puma(sample,y1,data_cols)

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [9]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)

In [10]:
df['inc_needed_sf'] = df['GEO_ID'].map(inc_needed_sf)
df['inc_needed_mf'] = df['GEO_ID'].map(inc_needed_mf)

In [11]:
dff = df[~(df.HHSz.isna())&(df.HINCP!=-60000)&(df.HINCP!=0)].copy()
dff['HINCP'] = dff.ADJINC * dff.HINCP #* inflation_adjust

In [12]:
dff['can_rent_sf'] = np.where(dff.HINCP>=dff.inc_needed_sf,'can rent','cannot afford')
dff['can_rent_mf'] = np.where(dff.HINCP>=dff.inc_needed_mf,'can rent','cannot afford')

In [13]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),inc_lbl[0],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),inc_lbl[1],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),
                   inc_lbl[2],np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),
                   inc_lbl[3],np.where((dff['HINCP']>dff['HHSz'].map(AMI_100pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_120pct)),
                                       inc_lbl[4],inc_lbl[5])))))

In [14]:
#make a column for each PUMA that assesses the number of households 
sf_cols=[]
for k in inc_needed_sf.keys():
    dff[f'aff_{k}_can_sf']=np.where(dff.HINCP>=inc_needed_sf[k],1,0)
    dff[f'aff_{k}_cant_sf']=np.where(dff.HINCP<inc_needed_sf[k],1,0)
    dff[f'aff_{k}_can_sf_wt']=dff[f'aff_{k}_can_sf']*dff.WGTP
    dff[f'aff_{k}_cant_sf_wt']=dff[f'aff_{k}_cant_sf']*dff.WGTP
    sf_cols = sf_cols + [f'aff_{k}_can_sf',f'aff_{k}_cant_sf',
                        f'aff_{k}_can_sf_wt',f'aff_{k}_cant_sf_wt']
    #sf_cols.append(f'aff_{k}_can_sf')
    #sf_cols.append(f'aff_{k}_cant_sf')
    #sf_cols.append(f'aff_{k}_can_sf_wt')
    #sf_cols.append(f'aff_{k}_cant_sf_wt')

In [15]:
#make a column for each PUMA that assesses the number of households 
mf_cols=[]
for k in inc_needed_mf.keys():
    dff[f'aff_{k}_can_mf']=np.where(dff.HINCP>=inc_needed_mf[k],1,0)
    dff[f'aff_{k}_cant_mf']=np.where(dff.HINCP<inc_needed_mf[k],1,0)
    dff[f'aff_{k}_can_mf_wt']=dff[f'aff_{k}_can_mf']*dff.WGTP
    dff[f'aff_{k}_cant_mf_wt']=dff[f'aff_{k}_cant_mf']*dff.WGTP
    mf_cols = mf_cols+[f'aff_{k}_can_mf',f'aff_{k}_cant_mf',
            f'aff_{k}_can_mf_wt',f'aff_{k}_cant_mf_wt']

/var/folders/s1/ytnwss3951g16cgxh_xr_rq00000gp/T/ipykernel_90006/3152188458.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff[f'aff_{k}_cant_mf']=np.where(dff.HINCP<inc_needed_mf[k],1,0)
/var/folders/s1/ytnwss3951g16cgxh_xr_rq00000gp/T/ipykernel_90006/3152188458.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff[f'aff_{k}_can_mf_wt']=dff[f'aff_{k}_can_mf']*dff.WGTP
/var/folders/s1/ytnwss3951g16cgxh_xr_rq00000gp/T/ipykernel_90006/3152188458.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

In [16]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,...,aff_0400125_can_mf_wt,aff_0400125_cant_mf_wt,aff_0400128_can_mf,aff_0400128_cant_mf,aff_0400128_can_mf_wt,aff_0400128_cant_mf_wt,aff_0400129_can_mf,aff_0400129_cant_mf,aff_0400129_can_mf_wt,aff_0400129_cant_mf_wt
3601,0400122,297649.1920,2.0,72.0,1.029928,69.0,19.0,68.0,74.0,21.0,...,72.0,0.0,1,0,72.0,0.0,1,0,72.0,0.0
3602,0400112,76214.6720,2.0,55.0,1.029928,56.0,96.0,91.0,53.0,16.0,...,55.0,0.0,1,0,55.0,0.0,0,1,0.0,55.0
3606,0400119,40476.1704,3.0,50.0,1.029928,51.0,87.0,16.0,92.0,87.0,...,0.0,50.0,0,1,0.0,50.0,0,1,0.0,50.0


### table by PUMA for renters by AMI range - can afford/can't afford median rent

In [17]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [18]:
drop_cols = ['HINCP','NP','ADJINC','HHSz','inc_needed_sf','inc_needed_mf']

In [19]:
table = dff.copy().drop(columns=drop_cols)

In [20]:
rent_ami = table.copy().drop(columns=sf_cols+mf_cols).groupby(['GEO_ID','AMI_range','can_rent_sf','can_rent_mf']).sum().reset_index()
rent_ami = make_est(rent_ami)
rent_ami = rent_ami.drop(columns=repwts)
rent_ami = pd.pivot_table(rent_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','can_rent_sf','can_rent_mf'],aggfunc=np.sum).reset_index()

In [21]:
rent_tot = table.copy().drop(columns=sf_cols+mf_cols+['AMI_range']).groupby(['GEO_ID','can_rent_sf','can_rent_mf']).sum().reset_index()
rent_tot = make_est(rent_tot)
rent_tot = rent_tot.drop(columns=repwts)
rent_tot = pd.pivot_table(rent_tot,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',
                          columns=['can_rent_sf','can_rent_mf'],aggfunc=np.sum).reset_index()

In [22]:
table_sf = table.copy().drop(columns=['AMI_range','can_rent_sf','can_rent_mf']+mf_cols).groupby(['GEO_ID']).sum().reset_index()
table_sf = make_est(table_sf)
table_sf = table_sf.drop(columns=repwts)

In [23]:
table_mf = table.copy().drop(columns=['AMI_range','can_rent_sf','can_rent_mf']+sf_cols).groupby(['GEO_ID']).sum().reset_index()
table_mf = make_est(table_mf)
table_mf = table_mf.drop(columns=repwts)

In [24]:
table_sf.head()

,GEO_ID,hh,aff_0400112_can_sf,aff_0400112_cant_sf,aff_0400112_can_sf_wt,aff_0400112_cant_sf_wt,aff_0400113_can_sf,aff_0400113_cant_sf,aff_0400113_can_sf_wt,aff_0400113_cant_sf_wt,...,aff_0400128_cant_sf,aff_0400128_can_sf_wt,aff_0400128_cant_sf_wt,aff_0400129_can_sf,aff_0400129_cant_sf,aff_0400129_can_sf_wt,aff_0400129_cant_sf_wt,hh_SE,hh_MOE,hh_CV
0,0400112,57425.0,326,305,29839.0,27586.0,410,221,38251.0,19174.0,...,186,41002.0,16423.0,420,211,39028.0,18397.0,1356.414391,2231.301673,1.435904
1,0400113,42824.0,224,272,18068.0,24756.0,281,215,23391.0,19433.0,...,189,26106.0,16718.0,285,211,23618.0,19206.0,1251.366213,2058.497421,1.776361
2,0400114,46724.0,129,355,12162.0,34562.0,185,299,17519.0,29205.0,...,264,21261.0,25463.0,197,287,18642.0,28082.0,1310.525143,2155.813860,1.705059
3,0400115,43853.0,86,308,9364.0,34489.0,138,256,14286.0,29567.0,...,234,15785.0,28068.0,145,249,14826.0,29027.0,1561.368302,2568.450857,2.164413
4,0400116,50270.0,143,410,10582.0,39688.0,214,339,17033.0,33237.0,...,315,19881.0,30389.0,221,332,17854.0,32416.0,1216.228371,2000.695670,1.470755


In [25]:
with pd.ExcelWriter(f'../../output/pums_gap_rent/pums_rent_gap_marketdata_120AMI_2BR.xlsx') as writer:
    rent_ami.to_excel(writer,sheet_name='can_cannot_withinpuma_ami.xlsx')
    rent_tot.to_excel(writer,sheet_name='can_cannot_withinpuma.xlsx')
    table_sf.to_excel(writer,sheet_name='can_cannot_all_sf.xlsx')
    table_mf.to_excel(writer,sheet_name='can_cannot_all_mf.xlsx')